In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import random
import networkx as nx
import matplotlib.pyplot as plt
import time
from collections import namedtuple
import pickle

from final_algo import (GET_ALGOS, get_algo_lambdas, kendall_cov, new_algo_KT, 
anandkumar_algo_lambda_wrapper_KT, stability_wrapper, glasso_vanilla_KT, SH_stability_wrapper_KT)
from paper_sims_util import omega_modularity
import sklearn
import sklearn.covariance

AlgoParams = namedtuple('AlgoParams', 'stability_samples M pi')
GraphParams = namedtuple('GraphParams', 'N eta p d ratios')

In [5]:
num_stocks = 100
run_id = 'kenall_tau_finance_p_{}'.format(num_stocks)

In [6]:
stocks = np.load('stockdata_data.npy')
stocks_log = np.diff(np.log(stocks), axis = 0)
print(stocks_log.shape)

info = np.load('stockdata_info.npy')
print(info.shape)
print(set(info[:, 1]))

sector = info[:num_stocks, 1]
S = np.cov(stocks_log[:, :num_stocks].T)
X = stocks_log[:, :num_stocks]
print(X.shape)

(1257, 452)
(452, 3)
{'Health Care', 'Materials', 'Consumer Discretionary', 'Information Technology', 'Energy', 'Telecommunications Services', 'Financials', 'Industrials', 'Consumer Staples', 'Utilities'}
(1257, 100)


In [7]:
sector_large = info[:, 1]
S_large = np.cov(stocks_log[:, :].T)
X_large = stocks_log[:, :]
print(X_large.shape)

(1257, 452)


In [21]:
#This is for CLIME and Tiger
from final_algo import tiger_algo, clime_algo
from final_algo import tiger_new_algo, clime_new_algo
tiger_res = tiger_algo(X)
#clime_res = clime_new_algo(X)

b'[1] "Loaded X"\n'
b'High-deimensional Sparse Undirected Graphical Models.\n'
b'Tuning-Insensitive Graph Estimation and Regression.\n'
b'[1] "DONE with tiger"\n'
b''


In [22]:
tiger_res.shape

(100, 100)

In [23]:
tiger_res

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [24]:
print(omega_modularity(tiger_res, sector))
print(omega_modularity(clime_res, sector))

-0.5
0


In [37]:
algo_params = AlgoParams(stability_samples = 100, M = 7./9., pi = 0.8)
graph_params = GraphParams(p=num_stocks, N = 1257, eta=1, ratios=None, d=None)
with open('{}_algo_params.pkl'.format(run_id), 'wb') as f:
    pickle.dump(algo_params, f)
with open('{}_graph_params.pkl'.format(run_id), 'wb') as f:
    pickle.dump(graph_params, f)
with open('{}_sector_X.pkl'.format(run_id), 'wb') as f:
    pickle.dump((sector, X), f)
    
algos = GET_ALGOS(algo_params.stability_samples)
#algo_lambdas = get_algo_lambdas(algo_params.M, graph_params.eta, graph_params.N, graph_params.p)
algo_lambdas = {
    'our': 7./9.,
    'SH': [0.7, 0.8, 0.9, 0.95, 0.99, 1],
    'nbsel':[0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'glasso':  [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1],
    'anand': [(x, 1) for x in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]]
}

KT_algos = {
    'our': new_algo_KT,
    'glasso': stability_wrapper(glasso_vanilla_KT, algo_params.stability_samples),
    'anand': anandkumar_algo_lambda_wrapper_KT,
    'SH': SH_stability_wrapper_KT(algo_params.stability_samples)
}
    
with open('{}_algo_lambdas.pkl'.format(run_id), 'wb') as f:
    pickle.dump(algo_lambdas, f)

np.random.seed(random.SystemRandom().randint(0, 2**32-2))
ALL_RESULTS = {}
for algo_name, algo in KT_algos.items():
    print("Currently on {}".format(algo_name))
    lambdas = algo_lambdas[algo_name]
    if algo_name == 'our':
        our_algo_res = algo(X, algo_params.M)
        results = (our_algo_res, None, None)
    elif algo_name == 'anand':
        lamb_to_res = algo(X, lambdas)
        for lamb, res in lamb_to_res.items():
            omega_hat = res[0]
            ALL_RESULTS[algo_name + str(lamb)] = res
            mod = omega_modularity(omega_hat, sector)
            print(lamb, mod)
    else:
        results = algo(X, lambdas, algo_params.pi)
    ALL_RESULTS[algo_name] = results
    with open('{}_{}_algo.pkl'.format(run_id, algo_name), 'wb') as f:
        pickle.dump(results, f)
    omega_hat = results[0]
    mod = omega_modularity(omega_hat, sector)
    print(mod)

Currently on our
Running KT VERSION new algorithm
N=1257, M=257
Working on l = 0


KeyboardInterrupt: 

In [12]:
glasso_cv = sklearn.covariance.GraphicalLassoCV(20)
#glasso_cv.fit(np.cov(X.T))
glasso_cv.fit(kendall_cov(X))
omega_hat = glasso_cv.get_precision()
non_zero = np.nonzero(omega_hat)
N, p = X.shape
A = np.zeros((p,p))
A[non_zero] = 1
print(glasso_cv.alpha_)
print(omega_modularity(A, sector))

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/site-packages/sklearn/covariance/graph_lasso_.py:240: RuntimeWarning: invalid value encountered in multiply
  * coefs)
/usr/local/lib/python3.6/site-packages/sklearn/covariance/graph_lasso_.py:242: RuntimeWarning: invalid value encountered in multiply
  * coefs)


3.4703355082314923e-09
0.1742955978861081


In [13]:
with open('first_finance_p_100_anand_algo.pkl', 'rb') as f:
    anand_result = pickle.load(f)
    omega_hat = anand_result[0]
    mod = omega_modularity(omega_hat, sector)
    print(mod)

-0.008814141414141318


In [12]:
mods = {}
for algo_name in algos.keys():
    print(algo_name)
    try:
        with open('{}_{}_algo.pkl'.format(run_id, algo_name), 'rb') as f:
            result = pickle.load(f)
    except:
        continue
    if type(result) is dict:
        result = next(iter(result.values()))
    omega_hat = result[0]
    mod = omega_modularity(omega_hat, sector)
    mods[algo_name] = mod
print(mods)

our
SH
glasso
nbsel
anand
{'SH': 0.41821131782205856, 'glasso': 0, 'nbsel': 0.3501047863710129}


In [181]:
from final_algo import anandkumar_algo_lambda_wrapper
anand_lambdas = [(1,x) for x in np.logspace(-6,1.2, num=20)]
print(anand_lambdas)
results, recon_info = anandkumar_algo_lambda_wrapper(X, anand_lambdas)

[(1, 1e-06), (1, 2.3930257311804994e-06), (1, 5.7265721500919525e-06), (1, 1.370383450663168e-05), (1, 3.279362859020883e-05), (1, 7.847599703514623e-05), (1, 0.00018779508018514912), (1, 0.00044939845907216705), (1, 0.001075422076112561), (1, 0.00257351270001691), (1, 0.006158482110660267), (1, 0.014737406155824811), (1, 0.03526699214174659), (1, 0.08439481965654007), (1, 0.20195897501643817), (1, 0.4832930238571752), (1, 1.1565326417902513), (1, 2.7676123707542306), (1, 6.622967617148325), (1, 15.848931924611133)]
Working on 1 1e-06
Working on 1 2.3930257311804994e-06
Working on 1 5.7265721500919525e-06
Working on 1 1.370383450663168e-05
Working on 1 3.279362859020883e-05
Working on 1 7.847599703514623e-05
Working on 1 0.00018779508018514912
Working on 1 0.00044939845907216705
Working on 1 0.001075422076112561
Working on 1 0.00257351270001691
Working on 1 0.006158482110660267
Working on 1 0.014737406155824811
Working on 1 0.03526699214174659
Working on 1 0.08439481965654007
Working o

In [182]:
for params, omega_hat in results.items():
    print(omega_modularity(omega_hat, sector))

-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008814141414141318
-0.008795782290459762
-0.008992033591761496
-0.008963865582473538
-0.009032287441955769
-0.009675122886049393
-0.00955680432202777
-0.008508482986767473
-0.008954113555868537


In [190]:
results_large, recon_info = anandkumar_algo_lambda_wrapper(X_large, anand_lambdas)

Working on 1 1e-06


KeyboardInterrupt: 